# Generate FPs for every Xchem fragment

In [1]:
import os
import pandas as pd
import pickle


def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [2]:
   def get_relevant_interaction(contact_dict, rel_interactions):
    
    #rel_interactions is either a str that says: "all" or a list of interactions
    
    #defined as relevant are the following interactions: (ONLY MAJOR)
    if rel_interactions == "all":
        relevant_interactions = ["Hydrogen Bond", "Weak Hydrogen Bond", 
                                 "Halogen Bond", "Ionic", "Aromatic", 
                                 "Hydrophobic", "Carbonyl", "Polar"]
    else:
        relevant_interactions = [rel_interactions]

    list_of_contacts = []
    for key,value in contact_dict.items():
        check = 0
        for relevant in relevant_interactions:
            if int(value[relevant]) == 1:
                check = 1
        if check == 1:
            list_of_contacts.append(key)
    return(list_of_contacts)


def make_interaction_maps(dict_df, relevant_interaction, type_of_contact, fragment):
    
    dict_subsite_residues = {}
    dict_subsite_atoms = {}
    dict_subsite_contacts = {}
    
    for frag in dict_df[fragment]:

        contact_dict = dict_df.loc[dict_df[fragment]==frag]["Contacts"].item()
        #print(contact_dict)
        relevant_contacts = get_relevant_interaction(contact_dict, str(relevant_interaction))

        #sort list and make second list with only protein atoms
        unique_prot_atoms = []
        unique_contacts_rearranged = []
        unique_prot_residues = []
        for i,contact in enumerate(relevant_contacts):
            partitioned = contact.partition("_")
            if partitioned[0].startswith("C"):
                pep = partitioned[0]
                prot = partitioned[2]
            else:
                pep = partitioned[2]
                prot = partitioned[0]


            unique_contacts_rearranged.append(pep + "---" + prot)
            unique_prot_atoms.append(prot)

            partitioned = prot.partition("/")
            part_part = partitioned[2].partition("/")
            res = partitioned[0]+partitioned[1]+part_part[0]

            unique_prot_residues.append(res)


        dict_subsite_contacts[frag] = unique_contacts_rearranged
        dict_subsite_atoms[frag] = list(dict.fromkeys(unique_prot_atoms))
        dict_subsite_residues[frag] = list(dict.fromkeys(unique_prot_residues))

    if type_of_contact == "full_contact":
        return dict_subsite_contacts
    if type_of_contact == "atoms":
        return dict_subsite_atoms
    if type_of_contact == "residues":
        return dict_subsite_residues

#functions to make the FPs
def generate_unique_res_list(dict_frags):
    all_residues = []
    for name, value in dict_frags.items():
        all_residues.extend([i for i in value])

    #make unique
    unique_residues = list(dict.fromkeys(all_residues))
    #sort by chain and ID
    unique_residues1 = [i for i in unique_residues if i[0]=="B"]
    unique_residues1.sort(key=lambda x: int(x.partition("/")[2].partition("/")[0]))

    unique_residues2 = [i for i in unique_residues if i[0]=="A"]
    unique_residues2.sort(key=lambda x: int(x.partition("/")[2].partition("/")[0]))

    return (unique_residues1+unique_residues2)

def construct_fp_matrix(contact_frags, unique_residues):
    
    df_residue_map = pd.DataFrame()

    for frag, residues in contact_frags.items():
        presence_list = []
        for prot_res in unique_residues:
            if prot_res in residues:
                presence_list.append(1)
            else:
                presence_list.append(0)
        df_residue_map[frag]=presence_list
    
    df_residue_map.index = unique_residues
    return df_residue_map.reindex(sorted(df_residue_map.columns), axis=1)

### Initialize folders and get contacts for frags

In [3]:
output_folder = "./clusters/"


pickled_contacts_folder = "./arpeggio_contacts/"

dict_df_frags = pd.DataFrame()
frags_list = []
contacts_list = []
for filename in os.listdir(pickled_contacts_folder):
    if filename.endswith("atoms.pkl"):
        temp_dict = load_obj(pickled_contacts_folder+filename)
        frags_list.append(filename[0:12])
        contacts_list.append(temp_dict)

dict_df_frags["Fragments"] = frags_list
dict_df_frags["Contacts"] = contacts_list

###### Generate the contact maps

In [4]:
all_contact_frags = make_interaction_maps(dict_df_frags, "all", "residues", "Fragments")

###### get a list of all protein residues involved in binding for all frags

In [5]:
unique_residues = generate_unique_res_list(all_contact_frags)

###### construct dataframe bitvector for interactions between every fragment and residue

In [6]:
df_residue_map_sorted = construct_fp_matrix(all_contact_frags, unique_residues)

## Calculate Tanimoto Similarity between those vectors

In [7]:
# to test if this is the same
from sklearn.metrics import jaccard_score
import numpy as np

In [8]:
#Initialzie Matrix
contact_matrix = pd.DataFrame(columns = df_residue_map_sorted.columns.tolist(), index = df_residue_map_sorted.columns.tolist())
#Fill the Matrix    
for col in contact_matrix.columns:
    for row in contact_matrix.index:
        temp_val = round(jaccard_score(df_residue_map_sorted[col][:].values,
                                                  df_residue_map_sorted[row][:].values), 2)
        contact_matrix[col][row] = temp_val
contact_matrix = contact_matrix[contact_matrix.columns].astype(float)

## Cluster by Tanimoto Sim

In [9]:
def cluster_similarity_matrix(contact_matrix, threshold):
    cluster_dict = {}
    cluster_counter = 0
    for ligand in contact_matrix:
        #print(ligand)
        #check if there are any duplicates in the clusters
        check_list = [item for key,item in cluster_dict.items()]
        check_list = [item for sublist in check_list for item in sublist]
        unique_check_list = list(dict.fromkeys(check_list))
        assert unique_check_list == check_list


        #generate list of all ligands with tan sim under the threshold
        sim_list = []
        for index, value in enumerate(contact_matrix[ligand]):
            if value >= threshold:
                sim_list.append(contact_matrix.index[index])

        if ligand not in check_list:
            #check if this lig is in a cluster alone 
            if len(sim_list) == 0:
                #print(ligand, "has no similar other ligs, making own cluster")
                cluster_counter += 1
                cluster_dict[cluster_counter] = [ligand]
                continue

            # make new cluster
            else:
                #print(ligand, "has similar other ligs, making new cluster")
                cluster_counter += 1
                #print(sim_list)
                sim_list = [i for i in sim_list if i not in check_list]
                #print(sim_list)
                cluster_dict[cluster_counter] = sim_list

        else:
            #print(ligand, "already in a cluster, updating that cluster")
            new_cluster_list = []
            new_sim_list = []
            for key,value in cluster_dict.items():
                if ligand in value:
                    local_cluster_counter = key

            for clust_lig in cluster_dict[local_cluster_counter]:
                if clust_lig in sim_list:
                    new_cluster_list.append(clust_lig)

            for sim_lig in sim_list:
                if sim_lig in new_cluster_list:
                    new_sim_list.append(sim_lig)

            temp_list = new_cluster_list + new_sim_list
            temp_list = list(dict.fromkeys(temp_list))
            #print(cluster_dict[local_cluster_counter], temp_list)
            cluster_dict[local_cluster_counter]
    return cluster_dict

In [10]:
contact_matrix

,Mpro-x0072_0,Mpro-x0104_0,Mpro-x0107_0,Mpro-x0161_0,Mpro-x0165_0,Mpro-x0177_0,Mpro-x0194_0,Mpro-x0195_0,Mpro-x0305_0,Mpro-x0336_0,...,Mpro-x1385_0,Mpro-x1386_0,Mpro-x1392_0,Mpro-x1402_0,Mpro-x1412_0,Mpro-x1418_0,Mpro-x1425_0,Mpro-x1458_0,Mpro-x1478_0,Mpro-x1493_0
Mpro-x0072_0,1.00,0.33,0.20,0.38,0.0,0.00,0.0,0.33,0.38,0.0,...,0.17,0.33,0.36,0.00,0.17,0.30,0.00,0.22,0.10,0.11
Mpro-x0104_0,0.33,1.00,0.27,0.62,0.0,0.00,0.0,0.56,0.44,0.0,...,0.14,0.40,0.31,0.00,0.14,0.36,0.00,0.08,0.08,0.09
Mpro-x0107_0,0.20,0.27,1.00,0.30,0.0,0.00,0.0,0.40,0.30,0.0,...,0.23,0.17,0.31,0.10,0.23,0.25,0.10,0.08,0.08,0.00
Mpro-x0161_0,0.38,0.62,0.30,1.00,0.0,0.00,0.0,0.62,0.50,0.0,...,0.07,0.30,0.23,0.00,0.07,0.27,0.00,0.09,0.00,0.00
Mpro-x0165_0,0.00,0.00,0.00,0.00,1.0,0.67,0.2,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mpro-x1418_0,0.30,0.36,0.25,0.27,0.0,0.00,0.0,0.36,0.40,0.0,...,0.55,0.88,0.80,0.50,0.55,1.00,0.50,0.40,0.40,0.44
Mpro-x1425_0,0.00,0.00,0.10,0.00,0.0,0.00,0.0,0.00,0.00,0.0,...,0.44,0.38,0.40,1.00,0.44,0.50,1.00,0.43,0.43,0.50
Mpro-x1458_0,0.22,0.08,0.08,0.09,0.0,0.00,0.0,0.08,0.09,0.0,...,0.67,0.44,0.45,0.43,0.67,0.40,0.43,1.00,0.33,0.57
Mpro-x1478_0,0.10,0.08,0.08,0.00,0.0,0.00,0.0,0.08,0.09,0.0,...,0.50,0.30,0.45,0.43,0.50,0.40,0.43,0.33,1.00,0.57


In [11]:
cluster_dict = cluster_similarity_matrix(contact_matrix, threshold=0.5)
sorted_idx = sorted(cluster_dict, key=lambda x: len(cluster_dict[x]), reverse=True)
cluster_dict_sorted_05 = {}
counter = 1
for idx in sorted_idx:
    cluster = cluster_dict[idx]
    cluster_dict_sorted_05[counter] = cluster
    counter +=1

###### save the dictionary

In [12]:
import json

In [13]:
save_obj(cluster_dict_sorted_05, output_folder+"all_interactions_tanimoto_0.5_cluster")
with open(output_folder+'all_interactions_tanimoto_0.5_cluster.json', 'w') as fp:
    json.dump(cluster_dict_sorted_05, fp)

In [14]:
cluster_dict = cluster_similarity_matrix(contact_matrix, threshold=0.7)
sorted_idx = sorted(cluster_dict, key=lambda x: len(cluster_dict[x]), reverse=True)
cluster_dict_sorted_07 = {}
counter = 1
for idx in sorted_idx:
    cluster = cluster_dict[idx]
    cluster_dict_sorted_07[counter] = cluster
    counter +=1

In [15]:
save_obj(cluster_dict_sorted_07, output_folder+"all_interactions_tanimoto_0.7_cluster")
with open(output_folder+'all_interactions_tanimoto_0.7_cluster.json', 'w') as fp:
    json.dump(cluster_dict_sorted_07, fp)